# Toronto Neighborhood Exploration

In [87]:
import numpy as np 

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


## 1. Web Scrape Toronto Data

In [88]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'html.parser')
table = soup.find_all('tbody')[0] 

In [89]:
#ignore the header
table_rows = table.find_all('tr')[1:]
table_rows[1]

<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [90]:
l = []
for tr in table_rows:
    #Get list of table row values
    td = tr.find_all('td')
    row_data = [value.text for value in td]
    
    #remove the new line '\n' from the Neighborhood or third element 
    row_data[2] = row_data[2].rstrip('\n')
    
    #ignore neighborhoods that are not assigned
    if row_data[1] != 'Not assigned':
        pc = row_data[0]
        b = row_data[1]
        # if a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
        n = row_data[1] if row_data[2] == 'Not assigned' else row_data[2]
        
        d = {"Postal Code": pc,
             "Borough": b, 
             "Neighborhood": n}
        
        l.append(d)
    
df = pd.DataFrame(l)
df = df.groupby(['Postal Code','Borough'])['Neighborhood'].agg(', '.join).reset_index()
df.head()

Postal Code      Borough                            Neighborhood
0         M1B  Scarborough                          Rouge, Malvern
1         M1C  Scarborough  Highland Creek, Rouge Hill, Port Union
2         M1E  Scarborough       Guildwood, Morningside, West Hill
3         M1G  Scarborough                                  Woburn
4         M1H  Scarborough                               Cedarbrae

Download dataset with boroughs and their locations

In [92]:
!wget -q -O 'toronto_data.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


Merge locations with toronto data

In [93]:
pc_location = pd.read_csv('toronto_data.csv')
toronto_data = df.merge(pc_location, on = 'Postal Code')
toronto_data

Postal Code           Borough  \
0           M1B       Scarborough   
1           M1C       Scarborough   
2           M1E       Scarborough   
3           M1G       Scarborough   
4           M1H       Scarborough   
5           M1J       Scarborough   
6           M1K       Scarborough   
7           M1L       Scarborough   
8           M1M       Scarborough   
9           M1N       Scarborough   
10          M1P       Scarborough   
11          M1R       Scarborough   
12          M1S       Scarborough   
13          M1T       Scarborough   
14          M1V       Scarborough   
15          M1W       Scarborough   
16          M1X       Scarborough   
17          M2H        North York   
18          M2J        North York   
19          M2K        North York   
20          M2L        North York   
21          M2M        North York   
22          M2N        North York   
23          M2P        North York   
24          M2R        North York   
25          M3A        North York   
26          M3B        North York   
27          M3C        North York   
28          M3H        North York   
29          M3J        North York   
30          M3K        North York   
31          M3L        North York   
32          M3M        North York   
33          M3N        North York   
34          M4A        North York   
35          M4B         East York   
36          M4C         East York   
37          M4E      East Toronto   
38          M4G         East York   
39          M4H         East York   
40          M4J         East York   
41          M4K      East Toronto   
42          M4L      East Toronto   
43          M4M      East Toronto   
44          M4N   Central Toronto   
45          M4P   Central Toronto   
46          M4R   Central Toronto   
47          M4S   Central Toronto   
48          M4T   Central Toronto   
49          M4V   Central Toronto   
50          M4W  Downtown Toronto   
51          M4X  Downtown Toronto   
52          M4Y  Downtown Toronto   
53          M5A  Downtown Toronto   
54          M5B  Downtown Toronto   
55          M5C  Downtown Toronto   
56          M5E  Downtown Toronto   
57          M5G  Downtown Toronto   
58          M5H  Downtown Toronto   
59          M5J  Downtown Toronto   
60          M5K  Downtown Toronto   
61          M5L  Downtown Toronto   
62          M5M        North York   
63          M5N   Central Toronto   
64          M5P   Central Toronto   
65          M5R   Central Toronto   
66          M5S  Downtown Toronto   
67          M5T  Downtown Toronto   
68          M5V  Downtown Toronto   
69          M5W  Downtown Toronto   
70          M5X  Downtown Toronto   
71          M6A        North York   
72          M6B        North York   
73          M6C              York   
74          M6E              York   
75          M6G  Downtown Toronto   
76          M6H      West Toronto   
77          M6J      West Toronto   
78          M6K      West Toronto   
79          M6L        North York   
80          M6M              York   
81          M6N              York   
82          M6P      West Toronto   
83          M6R      West Toronto   
84          M6S      West Toronto   
85          M7A  Downtown Toronto   
86          M7R       Mississauga   
87          M7Y      East Toronto   
88          M8V         Etobicoke   
89          M8W         Etobicoke   
90          M8X         Etobicoke   
91          M8Y         Etobicoke   
92          M8Z         Etobicoke   
93          M9A         Etobicoke   
94          M9B         Etobicoke   
95          M9C         Etobicoke   
96          M9L        North York   
97          M9M        North York   
98          M9N              York   
99          M9P         Etobicoke   
100         M9R         Etobicoke   
101         M9V         Etobicoke   
102         M9W         Etobicoke   

                                          Neighborhood   Latitude  Longitude  
0                                       Rouge, Malvern  43.806686 -79.194353

Get the rows that only contain boroughs with 'Toronto' in it

In [94]:
toronto_data = toronto_data[toronto_data['Borough'].str.contains('Toronto')].reset_index()
toronto_data.Borough.unique()

array(['East Toronto', 'Central Toronto', 'Downtown Toronto',
       'West Toronto'], dtype=object)

In [95]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [96]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [97]:
CLIENT_ID = '0BV0V0GOBEIOVT0MRXMTB0KWJQ4XMJPQ1IZPT1MKJY225P24' # your Foursquare ID
CLIENT_SECRET = 'MVTHTCREBFUDB4UPFWWGNQS0CFL5XWR2ENXIZBPQPZDMOHTG' # your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0BV0V0GOBEIOVT0MRXMTB0KWJQ4XMJPQ1IZPT1MKJY225P24
CLIENT_SECRET:MVTHTCREBFUDB4UPFWWGNQS0CFL5XWR2ENXIZBPQPZDMOHTG


In [98]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [99]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=0BV0V0GOBEIOVT0MRXMTB0KWJQ4XMJPQ1IZPT1MKJY225P24&client_secret=MVTHTCREBFUDB4UPFWWGNQS0CFL5XWR2ENXIZBPQPZDMOHTG&v=20180604&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [100]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e63eb659da7ee001b59693c'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'labe

In [101]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [102]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

In [103]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

name         categories        lat        lng
0                   Glen Manor Ravine              Trail  43.676821 -79.293942
1  The Big Carrot Natural Food Market  Health Food Store  43.678879 -79.297734
2                 Grover Pub and Grub                Pub  43.679181 -79.297215
3                       Upper Beaches       Neighborhood  43.680563 -79.292869
4                 Seaspray Restaurant   Asian Restaurant  43.678888 -79.298167

In [104]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


In [105]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [106]:
# type your answer here

toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )



The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction Sout

In [107]:
print(toronto_venues.shape)
toronto_venues.head()

(1728, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  The Beaches              43.676357              -79.293031   
1  The Beaches              43.676357              -79.293031   
2  The Beaches              43.676357              -79.293031   
3  The Beaches              43.676357              -79.293031   
4  The Beaches              43.676357              -79.293031   

                                Venue  Venue Latitude  Venue Longitude  \
0                   Glen Manor Ravine       43.676821       -79.293942   
1  The Big Carrot Natural Food Market       43.678879       -79.297734   
2                 Grover Pub and Grub       43.679181       -79.297215   
3                       Upper Beaches       43.680563       -79.292869   
4                 Seaspray Restaurant       43.678888       -79.298167   

      Venue Category  
0              Trail  
1  Health Food Store  
2                Pub  
3       Neighborhood  
4   Asian Restaurant

In [108]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Adelaide, King, Richmond                                              100   
Berczy Park                                                            56   
Brockton, Exhibition Place, Parkdale Village                           24   
Business Reply Mail Processing Centre 969 Eastern                      18   
CN Tower, Bathurst Quay, Island airport, Harbou...                     17   
Cabbagetown, St. James Town                                            47   
Central Bay Street                                                     79   
Chinatown, Grange Park, Kensington Market                              86   
Christie                                                               18   
Church and Wellesley                                                   85   
Commerce Court, Victoria Hotel                                        100   
Davisville                                                             34   
Davisville North                                                        9   
Deer Park, Forest Hill SE, Rathnelly, South Hil...                     15   
Design Exchange, Toronto Dominion Centre                              100   
Dovercourt Village, Dufferin                                           19   
First Canadian Place, Underground city                                100   
Forest Hill North, Forest Hill West                                     4   
Harbord, University of Toronto                                         36   
Harbourfront                                                           48   
Harbourfront East, Toronto Islands, Union Station                     100   
High Park, The Junction South                                          23   
Lawrence Park                                                           4   
Little Portugal, Trinity                                               56   
Moore Park, Summerhill East                                             4   
North Toronto West                                                     25   
Parkdale, Roncesvalles                                                 14   
Queen's Park                                                           43   
Rosedale                                                                4   
Roselawn                                                                1   
Runnymede, Swansea                                                     35   
Ryerson, Garden District                                              100   
St. James Town                                                        100   
Stn A PO Boxes 25 The Esplanade                                        95   
Studio District                                                        43   
The Annex, North Midtown, Yorkville                                    22   
The Beaches                                                             5   
The Beaches West, India Bazaar                                         18   
The Danforth West, Riverdale                                           41   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Adelaide, King, Richmond                                               100   
Berczy Park                                                             56   
Brockton, Exhibition Place, Parkdale Village                            24   
Business Reply Mail Processing Centre 969 Eastern                       18   
CN Tower, Bathurst Quay, Island airport, Harbou...                      17   
Cabbagetown, St. James Town                                             47   
Central Bay Street                                                      79   
Chinatown, Grange Park, Kensington Market                               86   
Christie                                                                18   
Church and Wellesley                

In [109]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 237 uniques categories.


## 3. Analyze each Neighborhood

In [110]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(columns = ['Neighborhood'], inplace = True)

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Neighborhood  Afghan Restaurant  Airport  Airport Food Court  Airport Gate  \
0  The Beaches                  0        0                   0             0   
1  The Beaches                  0        0                   0             0   
2  The Beaches                  0        0                   0             0   
3  The Beaches                  0        0                   0             0   
4  The Beaches                  0        0                   0             0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Antique Shop  Aquarium  Art Gallery  Arts & Crafts Store  Asian Restaurant  \
0             0         0            0                    0                 0   
1             0         0            0                    0                 0   
2             0         0            0                    0                 0   
3             0         0            0                    0                 0   
4             0         0            0                    0                 1   

   Athletics & Sports  Auto Workshop  BBQ Joint  Baby Store  Bagel Shop  \
0                   0              0          0           0           0   
1                   0              0          0           0           0   
2                   0              0          0           0           0   
3                   0              0          0           0           0   
4                   0              0          0           0           0   

   Bakery  Bank  Bar  Baseball Stadium  Basketball Stadium  Beach  \
0       0     0    0                 0                   0      0   
1       0     0    0                 0                   0      0   
2       0     0    0                 0                   0      0   
3       0     0    0                 0                   0      0   
4       0     0    0                 0                   0      0   

   Bed & Breakfast  Beer Bar  Beer Store  Belgian Restaurant  Bistro  \
0                0         0           0                   0       0   
1                0         0           0                   0       0   
2                0         0           0                   0       0   
3                0         0           0                   0       0   
4                0         0           0                   0       0   

   Boat or Ferry  Bookstore  Boutique  Brazilian Restaurant  Breakfast Spot  \
0              0          0         0                     0               0   
1              0          0         0                     0               0   
2              0          0         0                     0               0   
3              0          0         0                     0               0   
4              0          0         0                     0               0   

   Brewery  Bubble Tea Shop  Building  Burger Joint  Burrito Place  Bus Line  \
0        0                0         0             0              0         0   
1        0                0         0             0              0         0   
2        0                0         0             0              0         0   
3        0                0         0             0              0         0   
4        0                0         0             0              0         0   

   Business Service  Butcher  Café  Cajun / Creole Restaurant  Camera Store  \
0                 0        0     0                          0             0   
1                 0        0     0                          0             0   
2                 0        0     0                          0             0   


In [111]:
toronto_onehot.shape

(1728, 237)

In [112]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Neighborhood  Afghan Restaurant  \
0                            Adelaide, King, Richmond           0.000000   
1                                         Berczy Park           0.000000   
2        Brockton, Exhibition Place, Parkdale Village           0.000000   
3   Business Reply Mail Processing Centre 969 Eastern           0.000000   
4   CN Tower, Bathurst Quay, Island airport, Harbo...           0.000000   
5                         Cabbagetown, St. James Town           0.000000   
6                                  Central Bay Street           0.000000   
7           Chinatown, Grange Park, Kensington Market           0.000000   
8                                            Christie           0.000000   
9                                Church and Wellesley           0.011765   
10                     Commerce Court, Victoria Hotel           0.000000   
11                                         Davisville           0.000000   
12                                   Davisville North           0.000000   
13  Deer Park, Forest Hill SE, Rathnelly, South Hi...           0.000000   
14           Design Exchange, Toronto Dominion Centre           0.000000   
15                       Dovercourt Village, Dufferin           0.000000   
16             First Canadian Place, Underground city           0.000000   
17                Forest Hill North, Forest Hill West           0.000000   
18                     Harbord, University of Toronto           0.000000   
19                                       Harbourfront           0.000000   
20  Harbourfront East, Toronto Islands, Union Station           0.000000   
21                      High Park, The Junction South           0.000000   
22                                      Lawrence Park           0.000000   
23                           Little Portugal, Trinity           0.000000   
24                        Moore Park, Summerhill East           0.000000   
25                                 North Toronto West           0.000000   
26                             Parkdale, Roncesvalles           0.000000   
27                                       Queen's Park           0.000000   
28                                           Rosedale           0.000000   
29                                           Roselawn           0.000000   
30                                 Runnymede, Swansea           0.000000   
31                           Ryerson, Garden District           0.000000   
32                                     St. James Town           0.000000   
33                    Stn A PO Boxes 25 The Esplanade           0.000000   
34                                    Studio District           0.000000   
35                The Annex, North Midtown, Yorkville           0.000000   
36                                        The Beaches           0.000000   
37                     The Beaches West, India Bazaar           0.000000   
38                       The Danforth West, Riverdale           0.000000   

     Airport  Airport Food Court  Airport Gate  Airport Lounge  \
0   0.000000            0.000000      0.000000        0.000000   
1   0.000000            0.000000      0.000000        0.000000   
2   0.000000            0.000000      0.000000        0.000000   
3   0.000000            0.000000      0.000000        0.000000   
4   0.058824            0.058824      0.058824        0.117647   
5   0.000000            0.000000      0.000000        0.000000   
6   0.000000            0.000000      0.000000        0.000000   
7   0.000000            0.000000      0.000000        0.000000   
8   0.000000            0.000000      0.000000        0.000000   
9   0.000000            0.000000      0.000000        0.000000   
10  0.000000            0.000000      0.000000        0.000000   
11  0.000000            0.000000      0.000000        0.000000   
12  0.000000            0.000000      0.000000        0.000000   
13  0.000000            0.000000      0.000000        0.000000   
14  0.0000

In [113]:
toronto_grouped.shape

(39, 237)

In [114]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.07
1       Restaurant  0.05
2  Thai Restaurant  0.04
3             Café  0.04
4              Bar  0.03


----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1     Cheese Shop  0.04
2  Farmers Market  0.04
3          Bakery  0.04
4            Café  0.04


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0            Café  0.12
1          Bakery  0.08
2  Breakfast Spot  0.08
3     Coffee Shop  0.08
4   Grocery Store  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0           Yoga Studio  0.06
1         Auto Workshop  0.06
2            Smoke Shop  0.06
3               Brewery  0.06
4  Fast Food Restaurant  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Termin

In [115]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [128]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                           Adelaide, King, Richmond           Coffee Shop   
1                                        Berczy Park           Coffee Shop   
2       Brockton, Exhibition Place, Parkdale Village                  Café   
3  Business Reply Mail Processing Centre 969 Eastern           Yoga Studio   
4  CN Tower, Bathurst Quay, Island airport, Harbo...       Airport Service   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0            Restaurant       Thai Restaurant                  Café   
1    Seafood Restaurant                  Café          Cocktail Bar   
2                Bakery        Breakfast Spot           Coffee Shop   
3            Restaurant        Farmers Market         Auto Workshop   
4        Airport Lounge      Airport Terminal              Boutique   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0      Sushi Restaurant                   Bar                Lounge   
1              Beer Bar                Bakery        Farmers Market   
2           Yoga Studio               Stadium         Burrito Place   
3                   Spa         Burrito Place            Smoke Shop   
4       Harbor / Marina         Boat or Ferry                   Bar   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0        Cosmetics Shop            Steakhouse           Concert Hall  
1            Restaurant           Cheese Shop           Concert Hall  
2            Restaurant          Climbing Gym  Performing Arts Venue  
3      Recording Studio           Pizza Place             Skate Park  
4           Coffee Shop                 Plane       Sculpture Garden

In [129]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [130]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

index Postal Code          Borough                    Neighborhood  \
0     37         M4E     East Toronto                     The Beaches   
1     41         M4K     East Toronto    The Danforth West, Riverdale   
2     42         M4L     East Toronto  The Beaches West, India Bazaar   
3     43         M4M     East Toronto                 Studio District   
4     44         M4N  Central Toronto                   Lawrence Park   

    Latitude  Longitude  Cluster Labels 1st Most Common Venue  \
0  43.676357 -79.293031               1                 Trail   
1  43.679557 -79.352188               1      Greek Restaurant   
2  43.668999 -79.315572               1                  Park   
3  43.659526 -79.340923               1                  Café   
4  43.728020 -79.388790               2                  Park   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0      Asian Restaurant                   Pub     Health Food Store   
1           Coffee Shop    Italian Restaurant        Ice Cream Shop   
2                   Pub         Burrito Place            Steakhouse   
3           Coffee Shop             Gastropub                Bakery   
4                  Lake              Bus Line           Swim School   

    5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0     Distribution Center          Dessert Shop    Dim Sum Restaurant   
1  Furniture / Home Store    Frozen Yogurt Shop                   Pub   
2        Sushi Restaurant               Brewery  Fast Food Restaurant   
3                 Brewery    Italian Restaurant   American Restaurant   
4             Yoga Studio          Dessert Shop     Electronics Store   

         8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0                        Diner        Discount Store            Yoga Studio  
1                  Pizza Place          Liquor Store              Juice Bar  
2           Italian Restaurant         Movie Theater      Fish & Chips Shop  
3                         Park        Sandwich Place            Cheese Shop  
4  Eastern European Restaurant   Dumpling Restaurant             Donut Shop

In [131]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Cluster 1

In [132]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Postal Code  Longitude  Cluster Labels 1st Most Common Venue  \
22         M5N -79.416936               0                Garden   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
22           Yoga Studio      Department Store  Ethiopian Restaurant   

   5th Most Common Venue        6th Most Common Venue 7th Most Common Venue  \
22     Electronics Store  Eastern European Restaurant   Dumpling Restaurant   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
22            Donut Shop      Doner Restaurant                Dog Run

## Cluster 2

In [133]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Postal Code  Longitude  Cluster Labels 1st Most Common Venue  \
0          M4E -79.293031               1                 Trail   
1          M4K -79.352188               1      Greek Restaurant   
2          M4L -79.315572               1                  Park   
3          M4M -79.340923               1                  Café   
5          M4P -79.390197               1                  Park   
6          M4R -79.405678               1        Clothing Store   
7          M4S -79.388790               1          Dessert Shop   
9          M4V -79.400049               1           Coffee Shop   
11         M4X -79.367675               1           Coffee Shop   
12         M4Y -79.383160               1           Coffee Shop   
13         M5A -79.360636               1           Coffee Shop   
14         M5B -79.378937               1           Coffee Shop   
15         M5C -79.375418               1           Coffee Shop   
16         M5E -79.373306               1           Coffee Shop   
17         M5G -79.387383               1           Coffee Shop   
18         M5H -79.384568               1           Coffee Shop   
19         M5J -79.381752               1           Coffee Shop   
20         M5K -79.381576               1           Coffee Shop   
21         M5L -79.379817               1           Coffee Shop   
24         M5R -79.405678               1                  Café   
25         M5S -79.400049               1                  Café   
26         M5T -79.400049               1                   Bar   
27         M5V -79.394420               1       Airport Service   
28         M5W -79.374846               1           Coffee Shop   
29         M5X -79.382280               1           Coffee Shop   
30         M6G -79.422564               1         Grocery Store   
31         M6H -79.442259               1                Bakery   
32         M6J -79.419750               1                   Bar   
33         M6K -79.428191               1                  Café   
34         M6P -79.464763               1    Mexican Restaurant   
35         M6R -79.456325               1             Gift Shop   
36         M6S -79.484450               1           Coffee Shop   
37         M7A -79.389494               1           Coffee Shop   
38         M7Y -79.321558               1           Yoga Studio   

    2nd Most Common Venue   3rd Most Common Venue 4th Most Common Venue  \
0        Asian Restaurant                     Pub     Health Food Store   
1             Coffee Shop      Italian Restaurant        Ice Cream Shop   
2                     Pub           Burrito Place            Steakhouse   
3             Coffee Shop               Gastropub                Bakery   
5             Pizza Place          Breakfast Spot                   Gym   
6             Coffee Shop             Yoga Studio                   Spa   
7          Sandwich Place        Sushi Restaurant           Coffee Shop   
9                     Pub      Light Rail Station   American Restaurant   
11             Restaurant                    Café                   Pub   
12    Japanese Restaurant              Restaurant      Sushi Restaurant   
13                 Bakery                    Café                   Pub   
14         Clothing Store                    Café   Japanese Restaurant   
15                   Café              Restaurant                 Diner   
16     Seafood Restaurant                    Café          Cocktail Bar   
17     Italian Restaurant            Burger Joint        Sandwich Place   
18             Restaurant         Thai Restaurant                  Café   
19               Aquarium                    Café                 Hotel   
20                   Café              Restaurant                 Hotel   
21             Restaurant                    Café                 Hotel   
24         Sandwich Place             Coffee Shop           Pizza Place   
25              Bookstore                     Bar    Italian Restaurant   
26  Vie

## Cluster 3

In [134]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Postal Code  Longitude  Cluster Labels 1st Most Common Venue  \
4          M4N -79.388790               2                  Park   
8          M4T -79.383160               2                  Park   
10         M4W -79.377529               2                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
4                   Lake              Bus Line           Swim School   
8             Playground                 Trail           Summer Camp   
10            Playground                 Trail           Yoga Studio   

   5th Most Common Venue        6th Most Common Venue 7th Most Common Venue  \
4            Yoga Studio                 Dessert Shop     Electronics Store   
8          Deli / Bodega  Eastern European Restaurant   Dumpling Restaurant   
10         Deli / Bodega  Eastern European Restaurant   Dumpling Restaurant   

          8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
4   Eastern European Restaurant   Dumpling Restaurant             Donut Shop  
8                    Donut Shop      Doner Restaurant                Dog Run  
10                   Donut Shop      Doner Restaurant                Dog Run

## Cluster 4

In [135]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Postal Code  Longitude  Cluster Labels 1st Most Common Venue  \
23         M5P -79.411307               3         Jewelry Store   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
23                 Trail              Bus Line      Sushi Restaurant   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
23           Yoga Studio          Dessert Shop     Electronics Store   

          8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
23  Eastern European Restaurant   Dumpling Restaurant             Donut Shop